<a href="https://colab.research.google.com/github/Bazinga97/Recommendation-system-/blob/main/Books_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader# Path to the CSV file in Google Drive

books = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Books_recommendation_system/Books.csv", low_memory= False)
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Books_recommendation_system/Ratings.csv", low_memory= False)
users = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Books_recommendation_system/Users.csv", low_memory= False)



In [3]:
len(ratings)

1149780

In [4]:
ratings


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [62]:
ratings

tensor([ 0, 10, 10,  8], device='cuda:0')

In [5]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


## Label Encoders for user and item IDs

user_encoder.fit_transform(ratings['User-ID']):

fit: Learns the unique values of User-ID and assigns them a unique integer.
transform: Converts each User-ID to its corresponding integer.


item_encoder.fit_transform(ratings['ISBN']):

fit: Learns the unique values of ISBN and assigns them a unique integer.
transform: Converts each ISBN to its corresponding integer.

In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoders for user and item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# Fit and transform user and item IDs
ratings['user_id'] = user_encoder.fit_transform(ratings['User-ID'])
ratings['item_id'] = item_encoder.fit_transform(ratings['ISBN'])


# Fit and transform user and item IDs
ratings['user_id'] = user_encoder.fit_transform(ratings['User-ID'])
ratings['item_id'] = item_encoder.fit_transform(ratings['ISBN'])

In [8]:
ratings

,User-ID,ISBN,Book-Rating,user_id,item_id
0,276725,034545104X,0,104433,57188
1,276726,0155061224,5,104434,29750
2,276727,0446520802,0,104435,107392
3,276729,052165615X,3,104436,127253
4,276729,0521795028,6,104436,127287
...,...,...,...,...,...
1149775,276704,1563526298,9,104428,256876
1149776,276706,0679447156,0,104429,164861
1149777,276709,0515107662,10,104430,123711
1149778,276721,0590442449,10,104431,145625


Train_test split

In [9]:
# Calculate the split index
split_index = int(0.8 * len(ratings))  # 80% of the data for training

ratings_train = ratings[:split_index]
ratings_test = ratings[split_index:].reset_index(drop=True)

print(f"Training set size: {len(ratings_train)}")
print(f"Testing set size: {len(ratings_test)}")


Training set size: 919824
Testing set size: 229956


In [10]:
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'user_id': self.data['user_id'][idx],
            'book_id': self.data['item_id'][idx],
            'rating': self.data['Book-Rating'][idx]
        }

## **Rating Dataset : **

1. Encapsulation of rating data for accessing individual samples
2. Data loader for efficient loading of data for parallel processing
3. Train_Test split
4. Implements batch processing with specified barch sizes
5. Establishing robust data pipeline for data prepration and model evaluation



In [11]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Instantiate train and test datasets
train_dataset = RatingDataset(ratings_train)
test_dataset = RatingDataset(ratings_test)

# Create train and test loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Example usage of train_loader and test_loader
for batch in train_loader:
    # Process training batch
    pass

for batch in test_loader:
    # Process testing batch
    pass


In [12]:
print(len(train_dataset))
print(len(train_loader))

919824
14373


## GMF: Generalized Matrix factorization

Technique commonly used in collaborative_filtering based model:  

Matrix factorization : user-item interaction matrix is decomposed to lower-dimensional rep [Embeddings] for users and items.

Embedding capture : latent factors --> user preferences and item characterstics





Three arguments : users; items and embedding_size

ReLU() : rectified linear unit activation function : introduce non-linearity into the model--> for learning complex relationship between user and item embedding

Regularization: to prevent overfitting


output : sigmoid activation fucntion ensures the model's output is bounded between 0 and 1

In [13]:
class GMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(GMF, self).__init__()
        self.relu = nn.ReLU()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.fc = nn.Linear(embedding_size, 32)
        self.output_layer = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, user_ids, item_ids):
        user_embed = self.user_embedding(user_ids)
        item_embed = self.item_embedding(item_ids)
        element_product = user_embed * item_embed
        x = self.fc(element_product)
        x = self.relu(x)
        x = self.dropout(x)
        output = self.output_layer(x)
        output = torch.sigmoid(output)  # Ensure output is between 0 and 1
        return output.view(-1)

**Multi-layer perceptron**:

Connects user and item embeddings and then pass to fully connected layer to predict user-item interactions.

Model allows flexibility in defining number and size of hidden layer enabling experimenation with different architechture

In [14]:
class MLP(nn.Module):
    def __init__(self, num_users, num_items, embedding_size, hidden_layers=[64, 32]):
        super(MLP, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        layers = []
        input_size = embedding_size * 2
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            input_size = hidden_size
        layers.append(nn.Linear(hidden_layers[-1], 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids):
        user_embed = self.user_embedding(user_ids)
        item_embed = self.item_embedding(item_ids)
        concat_embed = torch.cat((user_embed, item_embed), dim=1)
        output = self.layers(concat_embed)
        output = torch.sigmoid(output)  # Ensure output is between 0 and 1
        return output.view(-1)

## Neural collaborative filtering  (NCF)


Captures non-linear relationships and complex interactions between users and items

Major bottleneck lies within traditional is data sparsity/ non-linear relationship ---> eliminated by NCF


In [15]:
class NCF(nn.Module):
    def __init__(self, gmf_model, mlp_model):
        super(NCF, self).__init__()
        self.gmf = gmf_model
        self.mlp = mlp_model

    def forward(self, user_ids, item_ids):
        gmf_output = self.gmf(user_ids, item_ids)
        mlp_output = self.mlp(user_ids, item_ids)
        combined_output = (gmf_output + mlp_output) / 2
        return combined_output

In [16]:
num_users = len(ratings['User-ID'].unique())
num_items = len(ratings['ISBN'].unique())
embedding_size = 64
hidden_layers = [128, 64, 32]

In [17]:
print(num_users)
print(num_items)

105283
340556


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [19]:
# Initialize GMF model
gmf_model = GMF(num_users, num_items, embedding_size).to(device)
# Initialize MLP model
mlp_model = MLP(num_users, num_items, embedding_size, hidden_layers).to(device)

Loss function: MSE {Mean squared error} --> to minimize the squared differences between predicted and actual values


Adam (Adaptive Moment Estimation) is an optimization algorithm that combines the advantages of two other extensions of stochastic gradient descent: AdaGrad and RMSProp. It is efficient, requires little memory, and is well-suited for problems with large data and parameters


The Adam optimizer is chosen for its ability to adapt the learning rate for each parameter, which helps in faster and more efficient convergence



In [20]:
#Loss criterion for GMF and MLP models
models_criterion = nn.MSELoss()

# Optimizer for GMF model
gmf_optimizer = optim.Adam(gmf_model.parameters(), lr=0.001)

# Optimizer for MLP model
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

In [21]:
def train_gmf(model, dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0
        i=0
        total_diff=0
        total_len=0
        for batch in dataloader:
            user_ids = batch['user_id'].to(device)
            item_ids = batch['book_id'].to(device)
            ratings = batch['rating'].to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, (ratings.float()/10) )
            loss.backward()
            optimizer.step()

            if (i % 1000 == 0):
                actual_ratings = 10*predictions
                diff = torch.abs( actual_ratings - ratings ).sum().item()
                print(f'Batch [{i+1}/{len(dataloader)}], Loss: {loss.item()}, Avg. Diff: { (diff/len(ratings)) }')

            i = i + 1

            total_loss += loss.item()

        print(f'GMF Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader)}')

In [22]:
def train_mlp(model, dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0
        i=0
        for batch in dataloader:
            user_ids = batch['user_id'].to(device)
            item_ids = batch['book_id'].to(device)
            ratings = batch['rating'].to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, (ratings.float()/10))
            loss.backward()
            optimizer.step()

            if (i % 1000 == 0):
                actual_ratings = 10*predictions
                diff = torch.abs( actual_ratings - ratings ).sum().item()
                print(f'Batch [{i+1}/{len(dataloader)}], Loss: {loss.item()}, Avg. Diff: { (diff/len(ratings)) }')

            i = i + 1

            total_loss += loss.item()

        print(f'MLP Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader)}')

In [23]:
num_epochs = 1

print("Training GMF...")
train_gmf(gmf_model, train_loader, models_criterion, gmf_optimizer, num_epochs)


Training GMF...
Batch [1/14373], Loss: 0.20354124903678894, Avg. Diff: 4.101387977600098
Batch [1001/14373], Loss: 0.1552814543247223, Avg. Diff: 3.6230270862579346
Batch [2001/14373], Loss: 0.1424468457698822, Avg. Diff: 3.43837571144104
Batch [3001/14373], Loss: 0.1350981891155243, Avg. Diff: 3.4560089111328125
Batch [4001/14373], Loss: 0.13463178277015686, Avg. Diff: 3.324162483215332
Batch [5001/14373], Loss: 0.12991094589233398, Avg. Diff: 3.2647337913513184
Batch [6001/14373], Loss: 0.1553080826997757, Avg. Diff: 3.566826343536377
Batch [7001/14373], Loss: 0.1405816376209259, Avg. Diff: 3.469696283340454
Batch [8001/14373], Loss: 0.12138818204402924, Avg. Diff: 3.213806629180908
Batch [9001/14373], Loss: 0.13776391744613647, Avg. Diff: 3.337874412536621
Batch [10001/14373], Loss: 0.15774297714233398, Avg. Diff: 3.5028438568115234
Batch [11001/14373], Loss: 0.13603007793426514, Avg. Diff: 3.239159345626831
Batch [12001/14373], Loss: 0.1357783079147339, Avg. Diff: 3.387942552566528

In [24]:
num_epochs = 1
print("Training MLP...")
train_mlp(mlp_model, train_loader, models_criterion, mlp_optimizer, num_epochs)

Training MLP...
Batch [1/14373], Loss: 0.20020289719104767, Avg. Diff: 4.241572856903076
Batch [1001/14373], Loss: 0.14956650137901306, Avg. Diff: 3.3502674102783203
Batch [2001/14373], Loss: 0.14657074213027954, Avg. Diff: 3.2623581886291504
Batch [3001/14373], Loss: 0.10036276280879974, Avg. Diff: 2.7209157943725586
Batch [4001/14373], Loss: 0.14786666631698608, Avg. Diff: 3.2815818786621094
Batch [5001/14373], Loss: 0.10451431572437286, Avg. Diff: 2.786642551422119
Batch [6001/14373], Loss: 0.13946402072906494, Avg. Diff: 3.154163360595703
Batch [7001/14373], Loss: 0.13125361502170563, Avg. Diff: 3.0579285621643066
Batch [8001/14373], Loss: 0.15037158131599426, Avg. Diff: 3.208573818206787
Batch [9001/14373], Loss: 0.1189032718539238, Avg. Diff: 2.8808395862579346
Batch [10001/14373], Loss: 0.13671410083770752, Avg. Diff: 3.095248222351074
Batch [11001/14373], Loss: 0.09988699853420258, Avg. Diff: 2.5046029090881348
Batch [12001/14373], Loss: 0.11566048860549927, Avg. Diff: 2.830127

In [25]:
model = NCF(gmf_model, mlp_model)

In [26]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
import math

In [28]:
num_epochs = 10
for epoch in range(num_epochs):
    i=0
    for batch in train_loader:
        user_ids = batch['user_id'].to(device)
        item_ids = batch['book_id'].to(device)
        ratings = batch['rating'].to(device)

        optimizer.zero_grad()
        predictions = model(user_ids, item_ids)
        loss = criterion(predictions, (ratings.float()/10))
        loss.backward()
        optimizer.step()

        if (i % 1000 == 0):
            actual_ratings = 10*predictions
            rmse = math.sqrt(torch.square(actual_ratings-ratings).sum().item()/len(ratings))
            diff = torch.abs( actual_ratings - ratings ).sum().item()
            print(f'Batch [{i+1}/{len(train_loader)}], Loss: {loss.item()}, Avg. Diff: { (diff/len(ratings)) }, RMSE: {rmse}')


        i = i + 1


    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Batch [1/14373], Loss: 0.12848326563835144, Avg. Diff: 2.986419200897217, RMSE: 3.584456127416001
Batch [1001/14373], Loss: 0.1541985273361206, Avg. Diff: 3.1811540126800537, RMSE: 3.9268119711510128
Batch [2001/14373], Loss: 0.13101914525032043, Avg. Diff: 3.0302257537841797, RMSE: 3.61965653787047
Batch [3001/14373], Loss: 0.14107291400432587, Avg. Diff: 3.1322708129882812, RMSE: 3.755967287922558
Batch [4001/14373], Loss: 0.09058365225791931, Avg. Diff: 2.3000948429107666, RMSE: 3.009711758319069
Batch [5001/14373], Loss: 0.11436644196510315, Avg. Diff: 2.700413465499878, RMSE: 3.3818106984962757
Batch [6001/14373], Loss: 0.09052764624357224, Avg. Diff: 2.4326822757720947, RMSE: 3.0087811384186467
Batch [7001/14373], Loss: 0.12918180227279663, Avg. Diff: 2.850614070892334, RMSE: 3.5941870382185512
Batch [8001/14373], Loss: 0.09959342330694199, Avg. Diff: 2.316472291946411, RMSE: 3.1558425187053696
Batch [9001/14373], Loss: 0.11445623636245728, Avg. Diff: 2.6512179374694824, RMSE: 3.

In [29]:
model.eval()

total_loss = 0
total_diff = 0
total_examples = 0
total_squared_error = 0

with torch.no_grad():
    for batch in test_loader:
        user_ids = batch['user_id'].to(device)
        item_ids = batch['book_id'].to(device)
        ratings = batch['rating'].to(device)

        predictions = model(user_ids, item_ids)
        loss = criterion(predictions, (ratings.float() / 10))

        actual_ratings = 10 * predictions
        diff = torch.abs(actual_ratings - ratings).sum().item()

        total_loss += loss.item() * len(ratings)
        total_diff += diff
        total_examples += len(ratings)
        total_squared_error += torch.square(actual_ratings-ratings).sum().item()

avg_loss = total_loss / total_examples
avg_diff = total_diff / total_examples
rmse = math.sqrt(total_squared_error / total_examples)

print('Evalution Measures:')
print(f'Evaluation Loss: {avg_loss}, Average Difference: {avg_diff}, RMSE: {rmse}')


Evalution Measures:
Evaluation Loss: 0.24828793008515684, Average Difference: 4.290937597307991, RMSE: 4.9828498694073415
